[View in Colaboratory](https://colab.research.google.com/github/progwriter/pybatfish/blob/victor-path-analysis/Introduction_to_Path_Analysis.ipynb)

# Introduction to Path Analysis using Batfish

Analyzing the paths of packets through the network is one of the most common tasks that network engineers undertake. Typically, this analysis is performed by running `traceroute` between multiple sources and destinations. This process is highly complex even in a moderately-sized network. It also fails to provide strong assurance as only some of the source-destination pairs and some of the packets can be feasibly tested.  

Batfish makes path analysis extremely simple by providing 1) easy-to-use queries over a centralized view of the network; and 2) ability to reason comprehensively about entire spaces of packets and locations. Further,  it can perform this analysis proactively, that is, analyze the impact of configuration changes *before* they are pushed to the network. 

In this notebook, we will show you how to perform path analysis with Batfish. ![](https://ga-beacon.appspot.com/UA-100596389-3/open-source/pybatfish/jupyter_notebooks/intro-path-analysis?pixel&useReferer)

In [1]:
# Import packages and load questions
%run startup.py

/Users/dhalperin/git-workspace/pybatfish/pybatfish/client/commands.py:50: UserWarning: Pybatfish public API is being updated, note that API names and parameters will soon change.
  "Pybatfish public API is being updated, note that API names and parameters will soon change.")


## Setup: Initializing the Network and Snapshot

`SNAPSHOT_PATH` below can be updated to point to a custom snapshot directory. See the [Batfish instructions](https://github.com/batfish/batfish/wiki/Packaging-snapshots-for-analysis) for how to package data for analysis.

More example networks are available in the [networks](https://github.com/batfish/batfish/tree/master/networks) folder of the Batfish repository.

In [2]:
NETWORK_NAME = "example_network"
SNAPSHOT_NAME = "example_snapshot"

SNAPSHOT_PATH = "networks/example"

bf_set_network(NETWORK_NAME)
bf_init_snapshot(SNAPSHOT_PATH, name=SNAPSHOT_NAME, overwrite=True)

'example_snapshot'

The network snapshot that we initialized above is illustrated below. You can view or download the devices' configuration files [here](https://github.com/batfish/pybatfish/tree/master/jupyter_notebooks/networks/example).

![example-network](https://raw.githubusercontent.com/batfish/pybatfish/master/jupyter_notebooks/networks/example/example-network.png)

All of the information we will show you in this notebook is dynamically computed by Batfish based on the configuration files for the network devices.

---



## Batfish Smart Traceroute: Detailed analysis of path(s) of a flow

In this section, we will use the [`traceroute` question](https://pybatfish.readthedocs.io/en/latest/questions.html#pybatfish.question.bfq.traceroute) to find the path taken by AS3 core routers to reach the DNS Server (`host1`) in AS2. Traceroute has three (composite) parameters that you can specify, allowing for a variety of queries. We will focus on the two main ones:

* `startLocation` - where in the network the flow starts
* `headers` - [packet headers](https://pybatfish.readthedocs.io/en/latest/datamodel.html#pybatfish.datamodel.flow.HeaderConstraints) for the flow you are interested in tracing. This is **not** just limited to UDP or ICMP.

We want the trace to start from the `Loopback0` interface on `as3core1`, and we want to use the IP address of that interface as the source address. For this we set the `startLocation` to `as3core1[Loopback0]`. Batfish automatically chooses the IP address of `Loopback0` as the source IP.

Let's set the destination IP address of our virtual packet by specifying `dstIps='ofLocation(host1)'`. Batfish will automatically pick *one of the* IP addresses for `host1` as the destination IP address.

To run the query:

In [0]:
# start the traceroute from the Loopback0 interface of as3core1 to host1
headers = HeaderConstraints(dstIps='ofLocation(host1)')
tracert = bfq.traceroute(startLocation="as3core1[Loopback0]", headers=headers).answer().frame()

To pretty-print the traces in HTML use the `display_html` function. We will show you how to extract more detailed information below.

In [21]:
display_html(tracert)

The `Flow` column describes the packet being traced: it starts at `as3core1` using source IP  `3.10.1.1` (of `Loopback0`) and and destination IP `2.128.0.101`. By default, `bfq.traceroute` uses the standard UDP traceroute to destination port `33434`, and Batfish arbitrarily picks the lowest ephemeral source port of  `49152`.

In [14]:
tracert['Flow'][0]

Flow(dscp=0, dstIp='2.128.0.101', dstPort=33434, ecn=0, fragmentOffset=0, icmpCode=255, icmpVar=255, ingressInterface=None, ingressNode='as3core1', ingressVrf='default', ipProtocol='UDP', packetLength=512, srcIp='3.10.1.1', srcPort=49152, state='NEW', tag='BASE', tcpFlagsAck=0, tcpFlagsCwr=0, tcpFlagsEce=0, tcpFlagsFin=0, tcpFlagsPsh=0, tcpFlagsRst=0, tcpFlagsSyn=0, tcpFlagsUrg=0)

The `Trace` column contains the detailed information provided by Batfish about the paths through the network for each flow. Let's look in detail on the first path:

In [19]:
tracert['Traces'][0][0] # Get the trace for the first path of the first flow

Trace(disposition='DENIED_IN', hops=[Hop(node='as3core1', steps=[Step(detail=OriginateStepDetail(originatingVrf='default'), action='ORIGINATED'), Step(detail=RoutingStepDetail(routes=[{'network': '2.128.0.0/16', 'nextHopIp': '10.23.21.2', 'protocol': 'ibgp'}]), action='FORWARDED'), Step(detail=ExitOutputIfaceStepDetail(outputInterface='GigabitEthernet1/0', outputFilter=None, transformedFlow=None), action='TRANSMITTED')]), Hop(node='as3border1', steps=[Step(detail=EnterInputIfaceStepDetail(inputInterface='GigabitEthernet0/0', inputVrf='default', inputFilter=None), action='RECEIVED'), Step(detail=RoutingStepDetail(routes=[{'network': '2.128.0.0/16', 'nextHopIp': '10.23.21.2', 'protocol': 'bgp'}]), action='FORWARDED'), Step(detail=ExitOutputIfaceStepDetail(outputInterface='GigabitEthernet1/0', outputFilter=None, transformedFlow=None), action='TRANSMITTED')]), Hop(node='as2border2', steps=[Step(detail=EnterInputIfaceStepDetail(inputInterface='GigabitEthernet0/0', inputVrf='default', inputFilter='OUTSIDE_TO_INSIDE'), action='RECEIVED'), Step(detail=RoutingStepDetail(routes=[{'network': '2.128.0.0/24', 'nextHopIp': '2.34.101.4', 'protocol': 'ibgp'}, {'network': '2.128.0.0/24', 'nextHopIp': '2.34.201.4', 'protocol': 'ibgp'}]), action='FORWARDED'), Step(detail=ExitOutputIfaceStepDetail(outputInterface='GigabitEthernet1/0', outputFilter=None, transformedFlow=None), action='TRANSMITTED')]), Hop(node='as2core2', steps=[Step(detail=EnterInputIfaceStepDetail(inputInterface='GigabitEthernet0/0', inputVrf='default', inputFilter=None), action='RECEIVED'), Step(detail=RoutingStepDetail(routes=[{'network': '2.128.0.0/24', 'nextHopIp': '2.34.201.4', 'protocol': 'ibgp'}]), action='FORWARDED'), Step(detail=ExitOutputIfaceStepDetail(outputInterface='GigabitEthernet2/0', outputFilter=None, transformedFlow=None), action='TRANSMITTED')]), Hop(node='as2dist2', steps=[Step(detail=EnterInputIfaceStepDetail(inputInterface='GigabitEthernet0/0', inputVrf='default', inputFilter=None), action='RECEIVED'), Step(detail=RoutingStepDetail(routes=[{'network': '2.128.0.0/24', 'nextHopIp': '2.34.201.4', 'protocol': 'bgp'}]), action='FORWARDED'), Step(detail=ExitOutputIfaceStepDetail(outputInterface='GigabitEthernet2/0', outputFilter=None, transformedFlow=None), action='TRANSMITTED')]), Hop(node='as2dept1', steps=[Step(detail=EnterInputIfaceStepDetail(inputInterface='GigabitEthernet1/0', inputVrf='default', inputFilter=None), action='RECEIVED'), Step(detail=RoutingStepDetail(routes=[{'network': '2.128.0.0/24', 'nextHopIp': 'AUTO/NONE(-1l)', 'protocol': 'connected'}]), action='FORWARDED'), Step(detail=ExitOutputIfaceStepDetail(outputInterface='GigabitEthernet2/0', outputFilter=None, transformedFlow=None), action='TRANSMITTED')]), Hop(node='host1', steps=[Step(detail=EnterInputIfaceStepDetail(inputInterface='eth0', inputVrf='default', inputFilter='filter::INPUT'), action='DENIED')])])

This flow starts at `as3core1` and crosses from AS3 into AS2 via the border routers `as3border1` and `as2border2`; on `as2border2`, the flow is permitted by the inbound ACL `OUTSIDE_TO_INSIDE`.

Once inside AS2, the flow is forwarded through AS2's core and distribution servers to the department router. The flow does reach `host1`, but is blocked by that server iptables rule `filter::INPUT`.

The `TraceCount` column reports the total number of paths for each flow. In this example, the count `4` matches the four paths we saw in the `Traces` column. If you look at the full table output above, you can see that the four paths These correspond to ECMP inside AS2 across `as2core1`/`2` and `as2dist1`/`2`.

<small>Detail: `TraceCount` may not always match the `TracesColumn`, this may not always be the case: in networks with high ECMP, there may be hundreds or even thousands of traces even for a single flow, in which case the `Traces` column will produce fewer results.</small>



In [15]:
tracert['TraceCount'][0]

4

Note that compared to running traceroute on a router, Batfish is able to provide much more detail about the trace:

1. All active parallel paths between the source and destination
1. The reason why each hop in a path is taken (the specific routing entry that was matched)
1. All processing steps inside each hop on the path
1. All interfaces visited and filters encountered during the trace
1. The disposition of the flow for each path

Of course, the traces are Python objects that are programmatically accessible as well. To get the detailed information about the final hop of the first trace in pure Python form:

In [5]:
flow_num = 0 # flow number we want to see
path_num = 0 # path number we want to see
last_hop = tracert['Traces'][flow_num][path_num].hops[-1]
repr(last_hop)

"Hop(node='host1', steps=[Step(detail=EnterInputIfaceStepDetail(inputInterface='eth0', inputVrf='default', inputFilter='filter::INPUT'), action='DENIED')])"

What we see from the above output is that the packet was sent all the way to `host1`, but blocked at the host itself. In fact, the iptables filter on the inbound interface `eth0` blocked the packet.

## Abstract Path Analysis with Batfish Reachability

Batfish's smart traceroute allows you to find detailed information about all paths taken by a specified flow through the network, but what if you want to check **every** possible flow, without trying them all? Batfish's
[`reachability` question](https://pybatfish.readthedocs.io/en/latest/questions.html#pybatfish.question.bfq.reachability) 
allows you to do that.

This abstract analysis is very useful when you want to build a set of tests for the network to ensure security and reliability.

Let's start by constructing a query that checks if **any** flow from the IPv4 address space (`srcIps='0.0.0.0/0`) originating inside AS2 can reach `host1`. 
We specify the destination IPs using the `ofLocation` function (see [documentation](https://github.com/batfish/batfish/blob/master/questions/Parameters.md#ip-specifier) for more detail).

In [6]:
path = PathConstraints(startLocation="as2.*")
headers = HeaderConstraints(srcIps="0.0.0.0/0", dstIps="ofLocation(host1)")
reach = bfq.reachability(pathConstraints=path, headers=headers, actions="success").answer().frame()
display_html(reach)

As you can see, this query identified *some* flow entering the network at *each* `as2...` node destined for `host1` that would be delivered.
And in all cases, the flows all happened to be SSH traffic. However, this does not guarantee that all SSH traffic from AS2 is delivered, or that no other traffic is delivered.

For guarantees like that, we have to search for bugs: flows we expect to have an action but that have the **opposite** action.

Let's verify DNS is accessible inside of AS2 and blocked to external devices. For accessibility, to guarantee that all DNS traffic from AS2 to `host1` is *accepted*, we look for the opposite: *any* DNS traffic from AS2 to `host1` that is *dropped*. Here, if Batfish is able to find any flow that is blocked, this would indicate a bug.

In [7]:
path = PathConstraints(startLocation="as2.*")
headers = HeaderConstraints(dstIps="ofLocation(host1)", applications="SSH")
reach = bfq.reachability(pathConstraints=path, headers=headers, actions="failure").answer().frame()
display_html(reach)

,Flow,Traces,TraceCount


The fact that Batfish returned 0 flows guarantees that `host1` is reachable via DNS from everywhere within AS2. We've assured availability.

But let's also verify security: no DNS traffic from *outside* of AS2 can reach `host1`. We'll search for flows starting from the border interfaces (`GigabitEthernet0/0` on AS2 border routers), and relax the source IP address to be any valid IP (`0.0.0.0/0`).

<small>_Details_: We use the [`enter` function](https://github.com/batfish/batfish/blob/master/questions/Parameters.md#interface-specifier) to model that traffic is received on the interface rather than starting from a border router. If we did not relax source IP to `0.0.0.0/0`, only source IP addresses within the connected subnet of the border interfaces would be included in the search.</small>

In [8]:
path = PathConstraints(startLocation="enter(as2border.*[GigabitEthernet0/0])")
headers = HeaderConstraints(srcIps="0.0.0.0/0", dstIps="ofLocation(host1)", applications="DNS")
reach = bfq.reachability(pathConstraints=path, headers=headers, actions="success").answer().frame()
display_html(reach)

We found that the DNS server is **not** secure: external DNS traffic can reach `host1`! However, we did find where to look: in all likelihood, the `OUTSIDE_TO_INSIDE` ACL on the border router should be blocking more DNS traffic.

Let's perform a slightly relaxed security check, searching for any flows **accepted** by `host1` that are **not** DNS or SSH:

In [9]:
path = PathConstraints(startLocation="enter(as2border.*[GigabitEthernet0/0])")
headers = HeaderConstraints(srcIps="0.0.0.0/0", dstIps="ofLocation(host1)", ipProtocols="TCP,UDP", dstPorts="!22,!53")
reach = bfq.reachability(pathConstraints=path, headers=headers, actions="accepted").answer().frame()
display_html(reach)

,Flow,Traces,TraceCount


Sucess! No unauthorized flows (not SSH or DNS) will reach `host1`

## Multipath Consistency

Finally, we will demonstrate an **experimental** feature to detect reachability bugs in any network with *no* user input: the [`multipathconsistency`](https://pybatfish.readthedocs.io/en/latest/questions.html#pybatfish.question.bfq.multipathConsistency) check. This question will report multipath inconsistency: it will find *any* flows with multipath routing where some paths reach the destination and some paths fail.

In [26]:
multipath = bfq.multipathConsistency().answer().frame()
first_result = multipath.head(1)  # this check returns many results, just show 1
display_html(first_result)

,Flow,Traces,TraceCount
0,Src IP: 2.1.2.2Src Port: 0Dst IP: 2.1.2.1Dst Port: 23IP Protocol: TCPStart Location: as2core2,"ACCEPTED1. node: as2core2 ORIGINATED(default) FORWARDED(Routes: ospf [Network: 2.1.2.1/32, Next Hop IP:2.12.22.1]) TRANSMITTED(GigabitEthernet0/0)2. node: as2border2 RECEIVED(GigabitEthernet1/0) FORWARDED(Routes: ospf [Network: 2.1.2.1/32, Next Hop IP:2.12.21.2]) TRANSMITTED(GigabitEthernet2/0)3. node: as2core1 RECEIVED(GigabitEthernet1/0) ACCEPTED(InboundStep)ACCEPTED1. node: as2core2 ORIGINATED(default) FORWARDED(Routes: ospf [Network: 2.1.2.1/32, Next Hop IP:2.12.12.1]) TRANSMITTED(GigabitEthernet1/0)2. node: as2border1 RECEIVED(GigabitEthernet2/0) FORWARDED(Routes: ospf [Network: 2.1.2.1/32, Next Hop IP:2.12.11.2]) TRANSMITTED(GigabitEthernet1/0)3. node: as2core1 RECEIVED(GigabitEthernet0/0) ACCEPTED(InboundStep)DENIED_IN1. node: as2core2 ORIGINATED(default) FORWARDED(Routes: ospf [Network: 2.1.2.1/32, Next Hop IP:2.23.22.3]) TRANSMITTED(GigabitEthernet2/0)2. node: as2dist2 RECEIVED(GigabitEthernet0/0) FORWARDED(Routes: ospf [Network: 2.1.2.1/32, Next Hop IP:2.23.12.2]) TRANSMITTED(GigabitEthernet1/0)3. node: as2core1 DENIED(GigabitEthernet3/0: blocktelnet)DENIED_IN1. node: as2core2 ORIGINATED(default) FORWARDED(Routes: ospf [Network: 2.1.2.1/32, Next Hop IP:2.23.21.3]) TRANSMITTED(GigabitEthernet3/0)2. node: as2dist1 RECEIVED(GigabitEthernet1/0) FORWARDED(Routes: ospf [Network: 2.1.2.1/32, Next Hop IP:2.23.11.2]) TRANSMITTED(GigabitEthernet0/0)3. node: as2core1 DENIED(GigabitEthernet2/0: blocktelnet)",4


The above trace shows that traffic from `as2core2` to `as2core1` can take four paths: through either of the two border routers or through either distribution router. However, telnet traffic will be blocked for only two of these four paths: the ones that traverse the distribution layer.



## Wrap-up

This concludes the notebook. To recap, in this notebook we covered the foundational tasks for path analysis:

1. We performed a traceroute to check connectivity to `host1`
2. Analyzed detailed path & hop information for the traceroute
3. Explored a space of flows with the reachablity question and found a ACL bug that allows some external clients to reach the DNS server
4. Perfomed a security check that ensures that only SSH and DNS traffic can reach `host1`
5. Found multipath inconsistency in the network, for which only some paths result in successful communication

We hope you found this notebook useful and informative. Future notebooks will dive into more advanced topics ensuring planned configuration changes do not have unintended consequences. Stay tuned!

### Want to know more? 

Reach out to us through [Slack](https://join.slack.com/t/batfish-org/shared_invite/enQtMzA0Nzg2OTAzNzQ1LTUxOTJlY2YyNTVlNGQ3MTJkOTIwZTU2YjY3YzRjZWFiYzE4ODE5ODZiNjA4NGI5NTJhZmU2ZTllOTMwZDhjMzA) or [Github](https://github.com/batfish/batfish) to learn more, or send feedback.